In [16]:
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
import numpy as np
import pandas

df = pandas.read_excel("Input data.xlsx")
y = df['y_up'].to_numpy(dtype=float)
df = df.drop(['y_up', 'y_down'], axis=1)
df
y


array([0., 0., 1., ..., 0., 0., 0.])

In [17]:
# Standardize the dataset
x = df.to_numpy(dtype=float)
scaler = StandardScaler()
x = scaler.fit_transform(x)
x

array([[-0.95140445, -0.77119774, -1.40167411, ...,  2.32691655,
         0.03518696, -1.72966581],
       [-0.95663765, -0.81838364, -0.81621862, ..., -0.21056596,
         0.03518696, -1.7291517 ],
       [-0.95280851, -1.05904612,  0.50299573, ...,  2.24344673,
         0.03518696, -1.72863759],
       ...,
       [ 2.27970293, -0.16519809, -0.4315819 , ..., -0.72807884,
        -0.53540121,  1.73389031],
       [ 2.25900479, -0.6769311 , -0.62772882, ...,  0.85784773,
         2.02653969,  1.73594674],
       [ 2.23866985, -0.66585145, -0.62063376, ..., -1.12873398,
        -0.32998947,  1.73646085]])

In [18]:
# Step 2: Apply PCA
pca = PCA(n_components=36)
X_pca = pca.fit_transform(x)

print("original shape:   ", x.shape)
print("transformed shape:", X_pca.shape)
print("explained variance ratio:", pca.explained_variance_ratio_)

original shape:    (4113, 58)
transformed shape: (4113, 36)
explained variance ratio: [0.18347142 0.16548167 0.11176524 0.07071035 0.05073588 0.03977421
 0.03691413 0.02889651 0.02771614 0.02407093 0.02138354 0.01840944
 0.01770325 0.01710249 0.01579806 0.01427089 0.01317567 0.01260686
 0.01156106 0.0106204  0.00991628 0.00920336 0.00857581 0.00827396
 0.00798753 0.00738008 0.00701171 0.00646579 0.00505147 0.00490368
 0.00471088 0.0039477  0.00389131 0.00322634 0.00269572 0.0025521 ]


In [19]:
from torch.utils.data import Dataset, random_split, DataLoader
from torch import from_numpy
import pandas as pd
import numpy as np

class CustomDataset(Dataset):
    def __init__(self, X, y):
        self.X = X
        self.y = y

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]

# Convert features and labels into numpy arrays and then into PyTorch tensors
X_data = from_numpy(x.astype(np.float32))
y_data = from_numpy(y.astype(np.float32))

# Load your data into the custom dataset
dataset = CustomDataset(X_data, y_data)

# Determine the lengths of your train/test splits (here, 70%/30% split)
total_len = len(dataset)
train_len = int(0.85 * total_len)
test_len = total_len - train_len

# Split the dataset
train_set, test_set = random_split(dataset, [train_len, test_len])

# Create data loaders
train_loader = DataLoader(train_set, batch_size=32, shuffle=True)
test_loader = DataLoader(test_set, batch_size=32, shuffle=True)

In [20]:
import torch.nn.functional as F
from torch import nn

class Model1(nn.Module):
  def __init__(self, input_len:int, output_len:int):
    super().__init__()
    self.fc1 = nn.Linear(input_len, input_len // 2)
    self.fc2 = nn.Linear(input_len // 2,input_len // 4)
    self.fc3 = nn.Linear(input_len // 4,output_len)
    
  def forward(self, x):
    x = x.view(x.shape[0], -1)
    
    x = F.relu(self.fc1(x))
    x = F.relu(self.fc2(x))
    x = F.sigmoid(self.fc3(x))
    
    return x

model = Model1(len(x[0]), 1)

TypeError: object of type 'numpy.float64' has no len()

In [8]:
from torch import optim
import timeit as time
criterion = nn.BCELoss()
optimizer = optim.SGD(model.parameters(), lr=0.01)

num_epochs = 3000

for i in range(num_epochs):
    cum_loss = 0

    start = time.default_timer()
    
    for images, labels in train_loader:
        optimizer.zero_grad()
        output = model(images)
        loss = criterion(output, labels)
        loss.backward()
        optimizer.step()
        
        cum_loss += loss.item()
     
    stop = time.default_timer()
    print(f"run {i + 1}, Training loss: {cum_loss/len(train_loader)}, run on the CPU with time {stop - start}")

c:\ProgramData\anaconda3\lib\site-packages\torch\nn\functional.py:1960: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")


run 1, Training loss: 0.6925270465287295, run on the CPU with time 0.12607470000511967
run 2, Training loss: 0.6832746256481518, run on the CPU with time 0.06403849998605438
run 3, Training loss: 0.672013087164272, run on the CPU with time 0.07991709999623708
run 4, Training loss: 0.6572752741250125, run on the CPU with time 0.11211619997629896
run 5, Training loss: 0.6378598532893441, run on the CPU with time 0.08199119998607785
run 6, Training loss: 0.6148515977642753, run on the CPU with time 0.06364919999032281
run 7, Training loss: 0.5905910860408436, run on the CPU with time 0.06326399999670684
run 8, Training loss: 0.5664362254467877, run on the CPU with time 0.06290170000283979
run 9, Training loss: 0.5424927158789201, run on the CPU with time 0.06420210000942461
run 10, Training loss: 0.5189721554517746, run on the CPU with time 0.06911149999359623
run 11, Training loss: 0.49993431215936485, run on the CPU with time 0.07134860000223853
run 12, Training loss: 0.481047507036816,

In [10]:
len(test_loader)

20

In [14]:
import torch
# test
model.eval()

correct = 0
total = 0
threshold = 0.5  # set your decision threshold for the probabilities

# No need to track gradients for validation, saves memory and computations
with torch.no_grad():
    for data in test_loader:
        inputs, labels = data
        outputs = model(inputs)
        
        # Convert the model outputs into predicted classes by thresholding
        predicted = (outputs > threshold).float()
        
        # Reshape labels to match prediction shape and convert to float
        labels = labels.view(predicted.shape).float()
        
        total += labels.size(0)
        print(f"run correct: {(predicted == labels).sum().item()}")
        correct += (predicted == labels).sum().item()
print(f"total: {total}")
print(f"Correct: {correct}")
print('Test Accuracy: {} %'.format(100 * correct / total))

run correct: 55
run correct: 50
run correct: 46
run correct: 52
run correct: 48
run correct: 49
run correct: 53
run correct: 57
run correct: 47
run correct: 44
run correct: 49
run correct: 47
run correct: 46
run correct: 47
run correct: 47
run correct: 53
run correct: 52
run correct: 45
run correct: 44
run correct: 18
total: 617
Correct: 949
Test Accuracy: 153.80875202593194 %


c:\ProgramData\anaconda3\lib\site-packages\torch\nn\functional.py:1960: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")
